# This is a very simple code to generate a image from text.

## >>> Quick SetUp 

### Execute this cell to install dependencies

In [ ]:
# Module Stuff
%cd "~/Desktop/keras_onnx_diffusers"
!poetry install
!poetry update --quiet

## <<< End of Quick SetUp

---

### Keras_cv Diffuser

In [ ]:
import datetime
import random

import keras_cv
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from PIL.PngImagePlugin import PngInfo
from tensorflow import keras
import os

# Check if directory exists
if os.path.exists("images/"):
    print("You already have 'images/' directory!")
else:
    # Create directory
    os.makedirs("images/")

# Load Model
model = keras_cv.models.StableDiffusionV2(
    img_height=512, img_width=512, jit_compile=True
)

# Set Mix Presicion
keras.mixed_precision.set_global_policy("mixed_float16")
print("Using mixed precision.")

# Image Plot
def plot_images(images):
    plt.figure(figsize=(10, 10))
    for i in range(len(images)):
        ax = plt.subplot(1, len(images), i + 1)
        plt.imshow(images[i])
        plt.axis("off")
    plt.show()

# Negative Prompt
NP = ("NSFW,watermark,bad face,bad hand,bad face shape,extra hands,extra fingers,bad arm,extra arms,missing leg,detached arm,"
      "inverted hand,ugly,bad eyes,logo,worst quality,blurry,bad anatomy,awful,separated,unpleasant quality")

# Set Seed
RI = random.randint(1, 1000000)

# Text to Image
def text_to_image(model, text, img_count=1):
    text = "best quality,master piece,ultra detailed,4K quality," + text
    model = model
    img_count = img_count
    global image
    image = model.text_to_image(
        str(text),
        negative_prompt=NP,
        batch_size=img_count,
        num_steps=24,  # Number of iterations (controls image quality)
        unconditional_guidance_scale=8,
        seed=RI,
    )
    # Print Seed
    print("///////" "seed", RI, "//////")
    # Image Plot
    plot_images(image)
    # Save Image
    pnginfo = PngInfo()
    pnginfo.add_text("prompt", text)
    Image.fromarray(image[0]).save(
        "images/" + str(datetime.datetime.now().strftime("%Y%m%d%H%M%S%f")) + ".png",
        pnginfo=pnginfo,
    )
    print("***" "saved atimages/ " "***")

# Execution
text = input("Enter Prompt Here" " Use Coron ->, to separate words:")
cnt = int(input("Enter the Number of Images You Want to Create: "))
text_to_image(model, text, img_count=cnt)

---

### ONNX Diffusion

In [ ]:
import random
import datetime
from PIL import Image
from PIL.PngImagePlugin import PngInfo
import numpy as np
import matplotlib.pyplot as plt
from diffusers import StableDiffusionOnnxPipeline

# Height and Width
H = 512
W = 512

# Set Pipe
pipe = StableDiffusionOnnxPipeline.from_pretrained("model/stable_diffusion_onnx", provider="DmlExecutionProvider")

# Negative Prompt
NP = ("NSFW,watermark,bad face,bad hand,bad face shape,extra hands,extra fingers,bad arm,extra arms,missing leg,detached arm,"
      "inverted hand,ugly,bad eyes,logo,worst quality,blurry,bad anatomy,awful,separated,unpleasant quality")

# Text to Image
def text_to_image(text,num_images_per_prompt=1):
    text = "best quality,master piece,ultra detailed,4K quality,breathtaking,authentic" + text
    for _ in range(num_images_per_prompt):
        image = pipe(str(text),
                     negative_prompt = NP,
                     height= H, width= W,
                     num_inference_steps = 96,  # Number of iterations (controls image quality)
                     guidance_scale = 7.2,
                     eta = 0.1,
                     num_images_per_prompt = 1,
                    ).images[0]
    
        # Display each image
        plt.imshow(image)
        plt.axis('off')  # to hide the axis
        plt.show()

        # Save Image
        pnginfo = PngInfo()
        pnginfo.add_text('prompt', text)
        image.save("output/" + str(datetime.datetime.now().strftime('%Y%m%d%H%M%S%f')) + ".png", pnginfo=pnginfo)
        print("***"\
             " saved at 'output/' "\
             "***")
# Execution
text = input("Enter Prompt Here"\
             " Use Coron ->, to separate words:" )
cnt = int(input("Enter the Number of Images You Want to Create: "))
text_to_image(text,num_images_per_prompt=cnt)

### Appendex

#### Make an Image into Black and White

In [ ]:
from tkinter import Button, Tk, filedialog

import numpy as np
from PIL import Image


def select_image():
    filename = filedialog.askopenfilename(
        filetypes=[("Image files", "*.jpg *.jpeg *.png")]
    )
    print("Selected:", filename)

    global selected_image
    selected_image = filename


root = Tk()
selected_image = None

# ボタンを作成して配置する
button = Button(root, text="Select Image", command=select_image)
button.pack()

root.mainloop()


# Load Image
img = Image.open(selected_image)

# NumPy配列を読み込む
arr = np.array(img)

# NumPy配列をPillowのImageオブジェクトに変換する
img = Image.fromarray(np.uint8(arr))

# Convert / 画像を白黒に変換
img = img.convert("L")

# Save Image / 保存する
img.save("images/" + str(datetime.datetime.now().strftime("%Y%m%d%H%M%S%f")) + ".png")
print("saved at image/")

## For Further Learning, Check Out the Links Below

- [**Official** Doc About Prompting @DreamStudio](https://beta.dreamstudio.ai/prompt-guide)

- [About Text to Image Using Diffusers @Huggingface](https://huggingface.co/docs/diffusers/api/pipelines/stable_diffusion/text2img)

- [About keras_cv @GitGub](https://github.com/keras-team/keras-cv)

- [**Japanese** Doc About How to Use Poetry](https://cocoatomo.github.io/poetry-ja/cli/)

- [Find a Prompt Here @Lexica](https://lexica.art/)

### A Bit of Tips

#### Run this cell

In [ ]:
from IPython.display import display, HTML, Image
from PIL import Image
import numpy as np

display(HTML('<h4>Using Old (like 100 years old) Artists Name Is Useful</h4>'))

def plot_images(image_paths):
    images = [np.array(Image.open(path)) for path in image_paths]

    plt.figure(figsize=(10, 10))

    for i in range(len(images)):
        ax = plt.subplot(1, len(images), i + 1)
        plt.imshow(images[i])
        plt.axis("off")

    plt.show()
    
display(HTML('<h5>Here are a Couple of Samples Made by Using the Name of Viktor Vansnetsov</h5>'))
image_paths = ["samples/sample1.png","samples/sample2.png"]
plot_images(image_paths)

html = """
<html>
<head>
    <title>Interactive Div</title>
    <style>
        #content {
            cursor: pointer;
            color: white;
            background-color: #007bff;
            width: 600px;
            height: 25px;
            padding: 10px;
            text-align: center;
            border-radius: 5px;
        }

        #myframe {
            display: none;
            width: 600px;
            height: 800px;
        }
    </style>

    <script src="https://code.jquery.com/jquery-3.5.1.min.js"></script>
    <script>
        $(document).ready(function(){
            $("#content").click(function(){
                $("#myframe").toggle();
            });
        });
    </script>
</head>
<body>
    <div id="content">Click here to Check Viktor Vansnetsov's Wikipedia</div>
    <iframe id="myframe" src="https://en.wikipedia.org/wiki/Viktor_Vasnetsov"></iframe>
</body>
</html>
"""

display(HTML(html))
